<h2>HR analysis</h2>

In [ ]:
%matplotlib inline

#You need the following modues, if you don't have them, use pip install <module-name>
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import datetime

#Update your start and end dates here in yyyy-mm-dd format 
start = datetime.datetime.strptime("2021-03-17", "%Y-%m-%d")
end = datetime.datetime.strptime("2021-03-22", "%Y-%m-%d")

date_array = (start + datetime.timedelta(days=x) for x in range(0, (end-start).days))
 
day_list = []
for date_object in date_array:
    day_list.append(date_object.strftime("%Y-%m-%d"))
    
print("day range : ",day_list)

day range :  ['2021-03-17', '2021-03-18', '2021-03-19', '2021-03-20', '2021-03-21']


In [ ]:
df_all = pd.DataFrame()

In [ ]:
# Implicit Grant Flow
#Enter your OAuth token in place
access_token = "OAuth token here"

header = {'Authorization': 'Bearer {}'.format(access_token)}

for single_day in day_list:
    response = requests.get("https://api.fitbit.com/1/user/-/activities/heart/date/"+ single_day +"/1d/1sec/time/00:00/23:59.json", headers=header).json()
    df = pd.DataFrame(response['activities-heart-intraday']['dataset'])
    date = pd.Timestamp(single_day).strftime('%Y-%m-%d')
    df = df.set_index(pd.to_datetime(date + ' ' + df['time'].astype(str)))
    #print(df)
    df_all = df_all.append(df, sort=True)
    
del df_all['time']

In [ ]:
#Put the interval you want to take the average of the imported data from fitbit with 2-5 sec interval. Default 10 minute
summary_df = (df_all['value'].resample('10Min').mean())

In [ ]:
plt.rcParams["figure.figsize"]=20,8

# Heart rate data summary [10min avg] from start date[2021-03-18] to end date[2021-03-21] 
#if you are using matplotlib directly in python ( py file ) then use plt.plot(summary_df,kind='area')
summary_df.plot.area()

<h2>Sleep analysis</h2>

In [ ]:
access_token = "OAuth token here"

header = {'Authorization': 'Bearer {}'.format(access_token)}

response = requests.get("https://api.fitbit.com/1.2/user/-/sleep/date/2021-03-18/2021-03-21.json", headers=header).json()

In [ ]:
combined_list = []
for i in range(len(response['sleep'])):
    result = response['sleep'][i]['levels']['summary']
    nice_dict = {'day':response['sleep'][i]['dateOfSleep'], 'deep_minutes':response['sleep'][i]['levels']['summary']['deep']['minutes'], 'light_minutes':response['sleep'][i]['levels']['summary']['light']['minutes'], 'rem_minutes':response['sleep'][i]['levels']['summary']['rem']['minutes'], 'wake_minutes':response['sleep'][i]['levels']['summary']['wake']['minutes']}
    combined_list.append(nice_dict)

In [ ]:
sleep_df = pd.DataFrame(combined_list)

In [ ]:
#Plotting data
sleep_df.set_index('day').plot(kind = 'bar')